In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=65d5aa5d113c553598ed522c0e23309c5056b79c5658a761285c197642cc6cec
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import *

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [24]:
from pyspark import *

In [ ]:
spark = SparkSession.builder.appName("fifa").getOrCreate()

df = spark.read.csv("/content/FIFA2022 Final Conerted final.csv", header=True, inferSchema=True)
#df.printSchema()
df.show()


+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|
|     Uruguay|    Portugal|   11|              3|        40|   403|           73|   16|           2|        0|      

*Add column 'Group' and populate the values based on countries :*

In [107]:
def group(input):
  if input in ['Qatar', 'Ecuador', 'Senegal', 'Netherlands']:
    return 'Group A'
  if input in ['England', 'Iran', 'United States', 'Wales']:
    return 'Group B'
  if input in [ 'Argentina', 'Saudi Arabia', 'Mexico', 'Poland']:
    return 'Group C'
  if input in ['France', 'Australia', 'Denmark', 'Tunisia']:
    return 'Group D'
  if input in ['Spain', 'Costa Rica','Germany','Japan']:
    return 'Group E'
  if input in ['Belgium', 'Canada', 'Morocco', 'Croatia']:
    return 'Group F'
  if input in ['Brazil', 'Serbia', 'Switzerland', 'Cameroon']:
    return 'Group G'
  if input in ['Portugal', 'Ghana', 'Uruguay', 'South Korea']:
    return 'Group H'
  return 'Other'
sparkfn=udf(group,StringType())
df2=df.withColumn("Group",sparkfn(col("Team")))
df2.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|  Group|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|Group H|
|     Uruguay|    Portugal|   11|              3|        40|   403|           73|   

In [108]:
def nullrem(input):
  if input in ['NA']:
    return 0
  else:
    return input
sparkfn=udf(nullrem,IntegerType())
df3=df2.withColumn("For Extra",sparkfn(col("For Goal Extra")))
df3.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+---------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|  Group|For Extra|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+---------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|Group H|        0|
|     Uruguay|    Portugal|   11|           

In [109]:
def nullrem(input):
  if input in ['NA']:
    return 0
  else:
    return input
sparkfn=udf(nullrem,IntegerType())
df3=df3.withColumn("Against Extra",sparkfn(col("Against Goal Extra")))
df3.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+---------+-------------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|  Group|For Extra|Against Extra|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+---------+-------------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|Group H|        0|   

**Add column 'Total Goals' representing total goals scored in the match**

In [110]:
df3=df3.withColumn("Total Goals",col("Goals For")+col("Goals against")+col("For Extra")+col("Against Extra"))
df3.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+---------+-------------+-----------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|  Group|For Extra|Against Extra|Total Goals|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-------+---------+-------------+-----------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|               

 Find out total goals scored in every group

In [111]:
df4=df3.groupBy("Group").sum("Total Goals")
df4.show()

+-------+----------------+
|  Group|sum(Total Goals)|
+-------+----------------+
|Group H|              48|
|Group A|              30|
|Group E|              44|
|  Other|              32|
|Group D|              32|
|Group G|              42|
|Group F|              16|
|Group B|              35|
|Group C|              34|
+-------+----------------+



Which country committed more fouls during group stage matches?

In [112]:
df3.select('Team','Fouls').where(col('Stage')=='Group').groupBy('Team').agg(sum("Fouls").alias("Total Fouls")).orderBy('Total Fouls',ascending=False).collect()[0][0]

'Saudi Arabia'

Which team got more Yellow cards in a match? Who was the opponent?

In [116]:
df5=df3.select('Team','Opponent','Yellow Cards').orderBy(col('Yellow Cards'),ascending=False)
print(df5.collect()[0][0],df5.collect()[0][1])

Argentina Netherlands


Which team/s scored more goals in group stages?

In [121]:
df7=df3.groupBy("Team").agg(sum("Goals For").alias("Total Goals")).orderBy(desc("Total Goals"))
df7.collect()[0]["Team"]

'France'